In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scipy.sparse as sp
import numpy as np
from tensorflow.contrib.rnn import RNNCell
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from sklearn.metrics import mean_squared_error,mean_absolute_error
#import matplotlib.pyplot as plt
import time
time_start=time.time()

In [2]:
# Loading of Dataset-SZ Traffic
def load_sz_data():
    sz_adj = pd.read_csv('sz_adj.csv',header=None)
    adj = np.mat(sz_adj)
    sz_tf = pd.read_csv('sz_speed.csv')
    return sz_tf, adj

data, adj = load_sz_data()

In [3]:
print(data) # Time sequence data

          90217      90218      90219      90220      90221      90222  \
0      8.471971  18.455410  20.590635  15.345258   9.585218  21.501821   
1      7.807137  15.713816  27.523695  11.087895   9.455280  17.332246   
2      8.809457   8.979647  20.280394  16.523419   8.003314  15.789483   
3     51.590372  23.631243  20.224094  15.116459   6.642644  17.575806   
4     58.770433  20.437740  20.465606  14.820217  11.344404   0.000000   
5     58.289126  10.332738  25.331018  18.485616   2.028513  10.718488   
6     61.334449  20.818480  15.839392   8.768217  13.933326  13.866124   
7     58.903144   8.291826  44.043729  15.250859   0.000000   0.000000   
8     57.210441  25.765667  18.677565   5.066051   0.000000  12.645903   
9     59.559265  16.803983  20.314610  23.393083   2.612821   0.000000   
10    60.217867  26.773576  17.684078  17.735375  12.957171   7.404007   
11    56.336405  18.866552  27.502295  18.201105  23.973238  22.215358   
12    58.511395  15.802270  15.900551 

In [4]:
print(adj) #Adjacency Matrix of Data

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 1 0]]


In [5]:
print(data.shape)
time_len=data.shape[0] # Time sequence length
num_nodes=data.shape[1] #Number of Roads

(2976, 156)


In [6]:
# Setting the variables
output_dim=pre_len=3
seq_len=4
num_units=100
train_rate=0.8
batch_size=32

In [7]:
#Normalization : Traffic Speed Data

data1 =np.mat(data,dtype=np.float32)

max_value = np.max(data1)
data1  = data1/max_value
print(data1)

   

[[0.0980221  0.21353212 0.23823702 ... 0.19194394 0.37707424 0.        ]
 [0.09032986 0.18181142 0.31845367 ... 0.45834997 0.37705195 0.        ]
 [0.10192686 0.10389599 0.23464748 ... 0.3856561  0.43354756 0.        ]
 ...
 [0.37947276 0.141638   0.10453826 ... 0.13605069 0.1998609  0.        ]
 [0.39722532 0.18600048 0.121989   ... 0.16306393 0.16957766 0.        ]
 [0.38101357 0.14001252 0.10420388 ... 0.19054307 0.1845446  0.        ]]


In [8]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate) #2976 *0.8 =2380
    train_data = data[0:train_size] #  [0:2380]
    test_data = data[train_size:time_len] #[2380:2976]
    print(train_data.shape,'----->',test_data.shape)

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len): #(2380-4-1)=2375
        a = train_data[i: i + seq_len + pre_len] #[0:0+4+1] 
        trainX.append(a[0 : seq_len]) #a[0:4] 4 time * 156 roads
        trainY.append(a[seq_len : seq_len + pre_len]) #a[4:4+1] 1 time*156 
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
      
    trainX1 = np.array(trainX) 
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)
    testY1 = np.array(testY)
    return trainX1, trainY1, testX1, testY1

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)  
print('Train Test Split Details :')
print('Train x ----> ',len(trainX))
print('Train y ----> ',len(trainY))
print(trainX.shape)
print('Test x ----> ',len(testX))
print('Test y ----> ',len(testY))
print('Total Batch ----> ',totalbatch)
#print('\nTrain Sample Details :')
#print(trainX[0],'--->',trainY[0])


(2380, 156) -----> (596, 156)
Train Test Split Details :
Train x ---->  2373
Train y ---->  2373
(2373, 4, 156)
Test x ---->  589
Test y ---->  589
Total Batch ---->  74


In [9]:
def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(1 / (input_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) 
#weights =weight_variable_glorot(4, 64, name='weights')
#weights

In [10]:
class stgcnCell(RNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(stgcnCell, self).__init__(_reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(self.calculate_laplacian(adj))

    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    def calculate_laplacian(self,adj, lambda_max=1):  
        adj = self.normalized_adj(adj + sp.eye(adj.shape[0])) # normalisation(self identity matrix + adj)
        adj = sp.csr_matrix(adj) #compressed sparse matrix
        adj = adj.astype(np.float32)
        return self.sparse_to_tuple(adj)
    
    def normalized_adj(self,adj):
        adj = sp.coo_matrix(adj)
        degree = np.array(adj.sum(1)) # Degree Matrix row wise sum
        d_inv_sqrt = np.power(degree, -0.5).flatten() # D inv = Degree ^-0.5 
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt) #substitution of the 1D array degree in a 2D matrix diagonals
        normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # norm= D^-0.5 * adj * D^-0.5
        normalized_adj = normalized_adj.astype(np.float32) 
        return normalized_adj
     
    ''' def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) '''
    
    def sparse_to_tuple(self,mx):
        mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() #coordinate stacking row and column wise and transpose
        L = tf.SparseTensor(coords, mx.data, mx.shape) # mx.shape= (156,156)
        #print('shape ---->',mx.shape)
        return tf.sparse_reorder(L) #row major ordering
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.variable_scope(scope or "tgcn"):
            with tf.variable_scope("gates"):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.variable_scope("candidate"):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))#(-1,156,64)
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
        input_size = x_s.get_shape()[2].value
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for m in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            weights = weight_variable_glorot(input_size, output_size, name='weights')
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.get_variable(
                "biases", [output_size], initializer=tf.constant_initializer(bias, dtype=tf.float32))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

In [11]:
def STGCN(_X, _weights, _biases):
    ###
    cell_1 = stgcnCell(num_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,num_units])
        o = tf.reshape(o,shape=[-1,num_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states

In [12]:
inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
weights = {
    'out': tf.Variable(tf.random_normal([num_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}
print(type(inputs))
pred,ttts,ttto = STGCN(inputs, weights, biases)

Instructions for updating:
Colocations handled automatically by placer.
<class 'tensorflow.python.framework.ops.Tensor'>
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [13]:
y_pred = pred

In [14]:
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])

In [15]:
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)

In [16]:
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))

In [17]:
lr=0.001
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [18]:
variables = tf.global_variables()
training_epoch=1000
saver = tf.train.Saver(tf.global_variables()) #
#sess = tf.Session()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())
out = 'out/%s'%("STGCN")
#out = 'out/%s_%s'%(model_name,'perturbation')
path1 = '%s_%s_lr%r_batch%r_unit%r_seq%r_pre%r_epoch%r'%("STGCN","sz",lr,batch_size,num_units,seq_len,pre_len,training_epoch)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)

In [19]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var

In [20]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]

In [21]:
def acc(a,b):  
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    train_acc=1-F_norm
    return train_acc

In [ ]:
for epoch in range(training_epoch):
    print("Epoch ", epoch)
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)
        train_label=np.reshape(mini_label,[-1,num_nodes])
        

     # Test completely at every epoch
    print("Accuracy ----> ", evaluation(train_label,train_output)[2])
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})
    #train_label=np.reshape(trainY,[-1,num_nodes])
    #train_acc=acc(train_label,train_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, test_output)
    test_label1 = test_label * max_value#Inverse normalization
    test_output1 = test_output * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    print(mini_label.shape,train_output.shape)
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    if (epoch % 500 == 0):        
        saver.save(sess, path+'/model_100/TGCN_pre_%r'%epoch, global_step = epoch)
        
time_end = time.time()
print(time_end-time_start,'s')

Epoch  0
Accuracy ---->  0.15271830558776855
(32, 3, 156) (96, 156)
Iter:0 train_rmse:12.19 test_loss:2.871e+03 test_rmse:0.1443 test_acc:0.1311
Epoch  1
Accuracy ---->  0.40825730562210083
(32, 3, 156) (96, 156)
Iter:1 train_rmse:8.514 test_loss:1.427e+03 test_rmse:0.1017 test_acc:0.3876
Epoch  2
Accuracy ---->  0.5120466947555542
(32, 3, 156) (96, 156)
Iter:2 train_rmse:7.021 test_loss:993.4 test_rmse:0.08488 test_acc:0.489
Epoch  3
Accuracy ---->  0.537372350692749
(32, 3, 156) (96, 156)
Iter:3 train_rmse:6.656 test_loss:900.9 test_rmse:0.08082 test_acc:0.5134
Epoch  4
Accuracy ---->  0.547220766544342
(32, 3, 156) (96, 156)
Iter:4 train_rmse:6.515 test_loss:866.1 test_rmse:0.07925 test_acc:0.5229
Epoch  5
Accuracy ---->  0.5555484294891357
(32, 3, 156) (96, 156)
Iter:5 train_rmse:6.395 test_loss:837.4 test_rmse:0.07792 test_acc:0.5309
Epoch  6
Accuracy ---->  0.5636116564273834
(32, 3, 156) (96, 156)
Iter:6 train_rmse:6.279 test_loss:810.1 test_rmse:0.07664 test_acc:0.5386
Epoch  7

Accuracy ---->  0.6618645191192627
(32, 3, 156) (96, 156)
Iter:58 train_rmse:4.865 test_loss:527.9 test_rmse:0.06185 test_acc:0.6276
Epoch  59
Accuracy ---->  0.662730872631073
(32, 3, 156) (96, 156)
Iter:59 train_rmse:4.853 test_loss:525.9 test_rmse:0.06173 test_acc:0.6283
Epoch  60
Accuracy ---->  0.6635704338550568
(32, 3, 156) (96, 156)
Iter:60 train_rmse:4.841 test_loss:524.0 test_rmse:0.06161 test_acc:0.629
Epoch  61
Accuracy ---->  0.6643834114074707
(32, 3, 156) (96, 156)
Iter:61 train_rmse:4.829 test_loss:522.1 test_rmse:0.0615 test_acc:0.6297
Epoch  62
Accuracy ---->  0.6651695668697357
(32, 3, 156) (96, 156)
Iter:62 train_rmse:4.818 test_loss:520.3 test_rmse:0.0614 test_acc:0.6304
Epoch  63
Accuracy ---->  0.6659278273582458
(32, 3, 156) (96, 156)
Iter:63 train_rmse:4.807 test_loss:518.5 test_rmse:0.06129 test_acc:0.631
Epoch  64
Accuracy ---->  0.6666567027568817
(32, 3, 156) (96, 156)
Iter:64 train_rmse:4.796 test_loss:516.9 test_rmse:0.06119 test_acc:0.6316
Epoch  65
Accu

Accuracy ---->  0.6848170161247253
(32, 3, 156) (96, 156)
Iter:116 train_rmse:4.535 test_loss:470.8 test_rmse:0.05839 test_acc:0.6485
Epoch  117
Accuracy ---->  0.6850170791149139
(32, 3, 156) (96, 156)
Iter:117 train_rmse:4.532 test_loss:470.3 test_rmse:0.05835 test_acc:0.6487
Epoch  118
Accuracy ---->  0.6852203905582428
(32, 3, 156) (96, 156)
Iter:118 train_rmse:4.529 test_loss:469.7 test_rmse:0.05832 test_acc:0.6489
Epoch  119
Accuracy ---->  0.6854273974895477
(32, 3, 156) (96, 156)
Iter:119 train_rmse:4.526 test_loss:469.1 test_rmse:0.05828 test_acc:0.6491
Epoch  120
Accuracy ---->  0.6856386065483093
(32, 3, 156) (96, 156)
Iter:120 train_rmse:4.523 test_loss:468.5 test_rmse:0.05824 test_acc:0.6493
Epoch  121
Accuracy ---->  0.6858544647693634
(32, 3, 156) (96, 156)
Iter:121 train_rmse:4.52 test_loss:468.0 test_rmse:0.05821 test_acc:0.6495
Epoch  122
Accuracy ---->  0.6860753893852234
(32, 3, 156) (96, 156)
Iter:122 train_rmse:4.517 test_loss:467.4 test_rmse:0.05817 test_acc:0.64

Accuracy ---->  0.6960351765155792
(32, 3, 156) (96, 156)
Iter:173 train_rmse:4.373 test_loss:439.1 test_rmse:0.05637 test_acc:0.6606
Epoch  174
Accuracy ---->  0.6961670517921448
(32, 3, 156) (96, 156)
Iter:174 train_rmse:4.372 test_loss:438.7 test_rmse:0.05634 test_acc:0.6608
Epoch  175
Accuracy ---->  0.6962983906269073
(32, 3, 156) (96, 156)
Iter:175 train_rmse:4.37 test_loss:438.3 test_rmse:0.05632 test_acc:0.6609
Epoch  176
Accuracy ---->  0.6964294016361237
(32, 3, 156) (96, 156)
Iter:176 train_rmse:4.368 test_loss:437.9 test_rmse:0.05629 test_acc:0.6611
Epoch  177
Accuracy ---->  0.6965602338314056
(32, 3, 156) (96, 156)
Iter:177 train_rmse:4.366 test_loss:437.6 test_rmse:0.05627 test_acc:0.6612
Epoch  178
Accuracy ---->  0.6966908872127533
(32, 3, 156) (96, 156)
Iter:178 train_rmse:4.364 test_loss:437.2 test_rmse:0.05624 test_acc:0.6614
Epoch  179
Accuracy ---->  0.6968214213848114
(32, 3, 156) (96, 156)
Iter:179 train_rmse:4.362 test_loss:436.8 test_rmse:0.05622 test_acc:0.66

In [ ]:
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]

index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)
var.to_csv(path+'/newtest_result.csv',index = False,header = False)
#plot_result(test_result,test_label1,path)
#plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)

In [ ]:
print("Testing:")
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

In [ ]:
print("Training:")
rmse, mae, acc, r2_score, var_score = evaluation(train_label,train_output)
print('min_rmse:%r'%(rmse),
      'min_mae:%r'%(mae),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score)) 

In [ ]:
#inverse normalisation
print('min_rmse:%r'%(rmse*max_value),
      'min_mae:%r'%(mae*max_value),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))